1º Parte: Receber o arquivo

In [ ]:
### Receber o video no colab (Upload)
from google.colab import files
uploaded = files.upload() # Até ~100MB

Saving BATALHA DOS MEMES.mp4 to BATALHA DOS MEMES.mp4


In [ ]:
### Receber o video da database
# https://www.kaggle.com/datasets/kunwardeepak/youtube-trending-august-2020

2º Parte: Processar o arquivo (audio e frames)

In [ ]:
### Verificar a duração do video para restrições de frames e audio
!pip install ffmpeg
import ffmpeg
import subprocess

# Retorna a duração em segundos
def verificarDuracaoVideo(videoEntrada):
  try:
    commando = [
      'ffprobe',
      '-i', videoEntrada,
      '-show_entries', 'format=duration',
      '-v', 'quiet',
      '-of', 'csv=p=0'
    ]
    # Executar o comando
    resultado = subprocess.run(commando, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    stderr = resultado.stderr.decode('utf-8')
    if stderr:
      print(f"Erro do ffprobe: {stderr}")
    # Ler a duração
    duracao = float(resultado.stdout.decode('utf-8').strip())
    return duracao
  except Exception as e:
    print(f"Erro ao converter a duração: {e}")
    return None


videoEntrada = None
# Encontrar o video
import os
for arquivo in os.listdir():
  if arquivo.endswith(".mp4"):
    videoEntrada = arquivo
    break

if videoEntrada:
  # Verificar a duração do video
  duracaoVideo = verificarDuracaoVideo(videoEntrada)
  print(videoEntrada,duracaoVideo)
else:
  print('Nenhum video encontrado!')

In [ ]:
### Extrair o audio do video
!pip install ffmpeg-python

import ffmpeg
import subprocess

def extrair_audio(videoEntrada, localAudioSaida):
  try:
    subprocess.run([
      'ffmpeg', '-i', videoEntrada,
      '-vn', '-acodec', 'libmp3lame', '-ab', '192k', localAudioSaida
    ], check=False)
    print("Audio extraido!")
  except Exception as e:
    print(f"Erro: {e}")

videoEntrada = None
# Encontrar o video
import os
for arquivo in os.listdir():
  if arquivo.endswith(".mp4"):
    videoEntrada = arquivo
    break

if videoEntrada:
  localAudioSaida = videoEntrada.replace('.mp4', '.mp3')
  extrair_audio(videoEntrada, localAudioSaida)
else:
  print('Nenhum video encontrado!')

In [ ]:
### Se necessário, repartir o audio em partes menores
!pip install ffmpeg-python
import ffmpeg
import os

def dividirAudio(audioEntrada, localAudioSaida, tamanhoMaximo = None):
  # Se não existir -> criar a pasta
  if not os.path.exists(localAudioSaida):
    os.makedirs(localAudioSaida)
  # Tentar dividir o audio
  try:
    ffmpeg.input(audioEntrada).output(f"{localAudioSaida}/chunk%03d.mp3", f='segment', segment_time=tamanhoMaximo).run()
    print("Audio dividido!")
  except ffmpeg.Error as e:
    print(f"Erro: {e.stderr.decode()}")

# Encontrar o audio
audioEntrada = None
for arquivo in os.listdir():
  if arquivo.endswith(".mp3"):
    audioEntrada = arquivo
    break

if audioEntrada:
  dividirAudio(audioEntrada,'/content/Audio/', 2*60)
else:
  print('Nenhum audio encontrado!')

In [ ]:
### Extrair os frames do video
!pip install opencv-python
import cv2
import time
import os

def extrairFrames(videoEntrada, localFramesSaida):
  try:
    # Se não existir -> criar a pasta
    if not os.path.exists(localFramesSaida):
      os.makedirs(localFramesSaida)

    start_time = time.time() # Para o cronometro

    cap = cv2.VideoCapture(videoEntrada)
    frame_count = 0
    frame_counter = 0
    # Determinar intervalo da captura
    fps = 1
    duracaoMaximaVideo = 5*60
    intervaloExtracao = 1.0/fps
    if duracaoVideo > duracaoMaximaVideo:
      intervaloExtracao = duracaoVideo/(fps*duracaoMaximaVideo)
    # Video em si
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # Get timestamp in seconds

      # Salvar o frame a cada {intervaloExtracao}
      if current_time >= frame_count * intervaloExtracao:
          frame_filename = os.path.join(localFramesSaida, f"frame_{frame_counter:04}_time_{current_time:.2f}.png")
          cv2.imwrite(frame_filename, frame)
          frame_counter += 1
          frame_count += 1

      # Break para quando o video ter acabado
      if current_time > cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0:
          break

    cap.release()
    end_time = time.time() # Parar o cronometro
    print('Frames extraidos')
    print(f"Tempo de execução: {end_time - start_time: .1f} segundos")
    print(f"{(frame_counter+1)/(end_time-start_time): .2f} frames por segundo")
  except Exception as e:
    print(f"Erro: {e}")

# Encontrar o video
videoEntrada = None
for arquivo in os.listdir():
  if arquivo.endswith(".mp4"):
    videoEntrada = arquivo
    break

if videoEntrada:
  extrairFrames(videoEntrada, '/content/Frames/Original/')
else:
  print('Nenhum video encontrado!')

In [ ]:
### Processar os frames
!pip install Pillow
from PIL import Image

def processarFrames(localFrames):
  try:
    # Se não existir -> criar a pasta
    if not os.path.exists('/content/Frames/Processados/'):
        os.makedirs('/content/Frames/Processados/')
    # Tentar processar os frames
    for arquivo in os.listdir(localFrames):
      if arquivo.endswith('.png'):
        # Imagem recebida
        with Image.open(os.path.join(localFrames,arquivo)) as img:
          # Verificando tamanho da imagem
          width, height = img.size
          if width > 1280 or height > 720: # Reduzir tamanho
            # Seguir proporção para evitar distorção
            proporcao = width/height
            if proporcao > 1: # Paisagem
              img = img.resize((1280, int(1280/proporcao)), Image.Resampling.LANCZOS)
            else: # Retrato
              img = img.resize((int(720*proporcao), 720), Image.Resampling.LANCZOS)

            # Salvar imagem na nova pasta
            img.save(os.path.join('/content/Frames/Processados/', arquivo))
    print('Frames processados')
  except Exception as e:
    print(f"Erro: {e}")

# Tentar processar os frames
import os
if os.path.exists('/content/Frames/Original/'):
  processarFrames('/content/Frames/Original/')
else:
  print('Nenhum frame encontrado!')

In [ ]:
#temp
!clear
import os
print(os.listdir())
if os.path.exists('/content/Frames/Original/'):
  print(sorted(os.listdir('/content/Frames/Original/')))
  print(len(os.listdir('/content/Frames/Original/')),end='\n------\n')
if os.path.exists('/content/Frames/Processados/'):
  print(sorted(os.listdir('/content/Frames/Processados/')))
  print(len(os.listdir('/content/Frames/Processados/')),end='\n------\n')

3º Parte: Criar descrição e transcrição dos frames e áudio, respectivamente

In [ ]:
### Instalar as bibliotecas maiores
!pip install openai-whisper
!pip install ollama
!pip install torch transformers pillow

In [ ]:
### Transcrição do audio (Whisper x Vosk)
## Whisper é mais preciso, mas é mais caro
## Implementação inicial com whisper, se ficar inviável implementar Vosk

## Whisper: Tempo de instalação = 2m 20s

!pip install openai-whisper
import whisper

def transcreverAudio(audioEntrada):
  try:
    # Tentar transcrever o audio
    model = whisper.load_model("base")
    result = model.transcribe(audioEntrada, language='pt')
    print("Audio Extraido!")
    return result["text"]
  except Exception as e:
    print(f"Erro: {e}")
    return None

audioEntrada = None
for arquivo in os.listdir():
  if arquivo.endswith(".mp3"):
    audioEntrada = arquivo
    break

if audioEntrada:
  transcricaoAudio = None # Salvar para caso de erro
  transcricaoAudio = transcreverAudio(audioEntrada)
else:
  print('Nenhum audio encontrado!')

In [ ]:
### Descrição dos frames
## 135 frames em 14 minutos
## Média de 5~7 segundos por frame
!pip install torch transformers pillow

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import os

pastaFrames = "/content/Frames/Processados/" # Tentar usar os frames já processados

try:
    # Tenta configurar o modelo
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    # Descrição de 1 frame
    def descrever_frame(caminho_imagem):
        image = Image.open(caminho_imagem).convert("RGB")

        # Processa a imagem com o processador BLIP, retornando os tensores necessários para o modelo
        inputs = processor(image, return_tensors="pt")

        # Gera a descrição da imagem a partir do modelo BLIP (max_length=100 significa que a descrição terá no máximo 100 tokens)
        output = model.generate(**inputs, max_length=100)

        # Decodifica a saída do modelo e converte para uma string legível
        descricao = processor.decode(output[0], skip_special_tokens=True)
        return descricao

    # Lista com a descrição dos frames
    def descreverFramesVideo(localFrames = '/content/Frames/Processados/'):
      import time
      listaDescricaoFrames = [] # List para armazenar as descrições
      # Carregar vários frames
      start_time = time.time()
      for frame in sorted(os.listdir(localFrames)):
          if frame.endswith((".jpg", ".png")):
              caminho = os.path.join(localFrames, frame)
              descricao = descrever_frame(caminho)
              # Dict para também armazenar o frame (por consequencia o timestamp)
              infoFrame = {
                  'frame': frame,
                  'descricao': descricao,
              }
              listaDescricaoFrames.append(infoFrame)
      end_time = time.time()
      print('Descrição de frames criada!')
      print(f"{(end_time-start_time)/len(listaDescricaoFrames): .2f} segundos por frame")
      return listaDescricaoFrames

except Exception as e:
    print(f"Erro: {e}")

# Usar original se não foi necessário processar
if not os.path.exists(pastaFrames): # Frames processados não existem -> usar os originais
  pastaFrames = '/content/Frames/Original/'

if videoEntrada:
  listDescricoesFrames = None # Salvar para caso de erro
  listDescricoesFrames = descreverFramesVideo(pastaFrames)
else:
  print('Nenhum video encontrado!')

4º Parte: Criação do sumário

Tentativa usando Deepseek

In [ ]:
### Tentativa de criação de contexto, criar uma descrição entre dois frames vizinhos (deepseek)
######## INCOMPLETO, checar chaveAPI e analisaVizinhos
######## Arrumar a localização para usar a variavel descricoesFrames
!pip install requests
import requests
import json

# Configurar para o uso
chaveAPI = "" # Adicionar uma chave
url = "https://api.deepseek.ai/v1/analyze" # Alterar dependendo

# Request headers com autorização
headers = {
    "Authorization": f"Bearer {chaveAPI}",
    "Content-Type": "application/json"
}

# Fazer a analise entre a descrição de dois frames
def analisaVizinhosDeepseek(descricao1, descricao2):
  conteudo = {
      'descricao1': descricao1,
      'descricao2': descricao2,
  }
  response = requests.post(url, headers=headers, json=conteudo)
  # Verificando sucesso
  if response.status_code == 200:
      return response.json() # Tentar alterar para uma string
      # Encontrar o campo que armazena o texto criado
      #try:
      #  # Extrai a string do campo apropriado do JSON
      #  return response.json().get('texto', None)
      #except ValueError:
      #  return 'Erro na análise da resposta'
  else:
    print(f"Error: {response.status_code}, {response.text}")
    return None

# Criar a lista com as novas descrições
def descricaoFramesVizinhosDeepseek():
  listDescricoesVizinhos = []
  listDescricoes = descricaoFrames() # Local já está com valor padrão correto
  for i in range(0, len(listDescricoes) - 1, 2):
    resposta = analisaVizinhosDeepseek(listDescricoes[i]['descricao'], listDescricoes[i+1]['descricao'])
    if resposta:
      # Armazenar as informações dos frames e a nova descrição
      dictResposta = {
          'frame1': listDescricoes[i]['frame'],
          'frame2': listDescricoes[i+1]['frame'],
          'descricao': resposta,
      }
      listDescricoesVizinhos.append(resposta)
  # Com a lista criada:
  if len(listDescricoesVizinhos) == len(listDescricoes) - 1: # Todos os elementos tiveram a descrição criada
    print('Descrição contextualizada criada!')
    return listDescricoesVizinhos
  else: # Algum não foi criado
    return None

In [ ]:
### Sumarizar a parte visual do video (deepseek)
######## INCOMPLETO, checar chaveAPI
!pip install requests
import requests
import json

# Configurar para o uso
chaveAPI = "" # Adicionar uma chave
url = "https://api.deepseek.ai/v1/analyze" # Alterar dependendo

# Request headers com autorização
headers = {
    "Authorization": f"Bearer {chaveAPI}",
    "Content-Type": "application/json"
}

# Fazer a analise entre múltiplas descrições
def analisaDescricoesFramesDeepseek(descricoes):
    # Combine todas as descrições em um formato adequado para a API
    conteudo = {'descricoes': descricoes}

    response = requests.post(url, headers=headers, json=conteudo)

    # Verificando sucesso
    if response.status_code == 200:
        return response.json()  # Retorna a resposta JSON completa
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Criar a lista com as novas descrições
def analiseFramesDeepseek():
  listDescricoesVizinhos = []
  #listDescricoes = descricaoFrames()  # Local já está com valor padrão correto
  listDescricoes = descricaoFramesVizinhosDeepseek() # Tentar usar o texto contextualizado
  # Agrupar todas as descrições em uma lista única
  descricoes = [d['descricao'] for d in listDescricoes]
  # Chamar a função de análise para todas as descrições
  resposta = analisaDescricoesFramesDeepseek(descricoes)
  if resposta:
    print('Descrição contextualizada criada!')
  return resposta

In [ ]:
### Criação do sumário final, sumarizar com a transcrição do audio e descrição dos frames (deepseek)


Tentativa usando o ollama

In [ ]:
### Tentativa de criação de contexto, criar uma descrição entre dois frames vizinhos (LLaMa)
# Baseado em https://github.com/GoloMarcos/LLM4BrazilianFakeNews/blob/main/Evaluation.ipynb
!pip install ollama
import ollama

# Fazer a analise entre a descrição de dois frames
def analisaVizinhosLlama(model, desc1, desc2):
  descricao1 = desc1['descricao']
  frame1_time = desc1['frame'].split('_time_')[1].split('.')[0]
  descricao2 = desc2['descricao']
  frame2_time = desc2['frame'].split('_time_')[1].split('.')[0]

  response = ollama.chat(model=model, messages=[{
    'role': 'user',
    'content': f'Considere as duas descrições em inglês a seguir e resuma-as em pt-br, contextualizando elas em conjunto. Explique como as duas se relacionam/diferenciam e o que elas podem significar em conjunto\nDescrição 1: {descricao1} instante {frame1_time}\nDescrição 2: {descricao2} instante {frame2_time}\nSeu resumo deve integrar as duas descrições e proporcionar uma visão clara e coerente do contexto geral.'
  },
  ])
  return response['message']['content']

# Criar a lista com as novas descrições
model = 'llama3:8b'
def descricaoFramesVizinhosLlama(listDescricoesRecebida=None):
  listDescricoesVizinhos = []
  #listDescricoes = listDescricoesRecebida
  for i in range(0, len(listDescricoesFrames) - 1, 2):
    resposta = analisaVizinhosLlama(model, listDescricoesFrames[i], listDescricoesFrames[i+1])
    if resposta:
      # Armazenar as informações dos frames e a nova descrição
      dictResposta = {
          'frame1': listDescricoesFrames[i]['frame'],
          'frame2': listDescricoesFrames[i+1]['frame'],
          'descricao': resposta,
      }
      listDescricoesVizinhos.append(dictResposta)
  # Com a lista criada:
  if len(listDescricoesVizinhos) == len(listDescricoesFrames) - 1: # Todos os elementos tiveram a descrição criada
    print('Descrição contextualizada criada!')
    return listDescricoesVizinhos
  else:
    return None


if listDescricoesFrames:
  ## receber
  resultadoContexto = descricaoFramesVizinhosLlama(listDescricoesFrames)
else:
    print('Nenhuma descrição encontrada!')

In [ ]:
### Sumarizar a parte visual do video (LLaMa)
# Baseado em https://github.com/GoloMarcos/LLM4BrazilianFakeNews/blob/main/Evaluation.ipynb
!pip install ollama
import ollama

# Fazer a analise entre a descrição de dois frames
def analisaDescricoesFramesLlama(model, descricoes):
  # Criação do prompt para todas as descrições
  prompt = "Considere as seguintes descrições e resuma-as, contextualizando todas em conjunto. Explique como elas se relacionam/diferenciam e o que elas podem significar em conjunto:\n"

  # Adiconar cada descrição e sua respectiva timestamp
  for i, desc in enumerate(descricoes):
    #prompt += f"\nDescrição {i + 1}: {desc['descricao']} instante {desc['frame'].split('_time_')[1].split('.')[0]}"
    prompt += f"\nDescrição {i + 1}: {desc['descricao']} instante {desc['frame1'].split('_time_')[1].split('.')[0]} até instante {desc['frame2'].split('_time_')[1].split('.')[0]}"

  prompt += "\nSeu resumo deve integrar todas as descrições e fornecer uma visão clara e coerente do contexto geral."

  response = ollama.chat(model=model, messages=[{
    'role': 'user',
    'content': prompt,
  },
  ])
  return response['message']['content']

# Criar a lista com as novas descrições
model = 'llama3:8b'
def analiseFramesLlama():
  listDescricoesVizinhos = []
  #listDescricoes = descricaoFrames() # Local já está com valor padrão correto
  listDescricoes = descricaoFramesVizinhosLlama() # Tentar usar o texto contextualizado

  if listDescricoes:
    # Chama a função de análise para todas as descrições de uma vez
    resposta = analisaDescricoesFramesLlama('llama3:8b', listDescricoes)
    if resposta:
      print('Descrição final dos frames criada!')
    return resposta # Pode ser vazia
  else:
      return None

In [ ]:
### Criação do sumário final, sumarizar com a transcrição do audio e descrição dos frames (LLaMa)


Possíveis APIs para a 4ª parte: uso de BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies), Hugging Face Models

In [ ]:
# Q

5º Parte: Análise do sumário com dados do video